<a href="https://colab.research.google.com/github/AyushiKashyapp/NLP/blob/main/TextClassificationDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with TensorFlow

Text classification model with Tensorflow on movie reviews as positive or negative using the text of the reviews.

A basic application of transfer learning with TensorFlow Hub and Keras, using the IMDB dataset which contains the text of 50,000 movie reviews from the internet movie database. These are divided into 25,000 assessments for training and 25,000 assessments for testing. The training and test sets are balanced in a way that they contian an equal number of positive and negative reviews.

In [2]:
import numpy as np
import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


**Downloading the dataset available in TensorFlow.**

Splitting the training set into 60% and 40%, 15,000 examples for training, 10,000 examples for validation and 25,000 examples for testing.

In [3]:
import tensorflow_datasets as tfds


train_data, validation_data = tfds.load(
    name="imdb_reviews",
    split=['train[:60%]', 'train[60%:]'],
    as_supervised=True
)

test_data = tfds.load(
    name="imdb_reviews",
    split='test',
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.S3KVXX_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.S3KVXX_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.S3KVXX_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

**Building text classification model**

Building a text classification model with TensorFlow using TensorFlow Hub, which is a pre-trained model provided by TensorFlow.

Firstly, creating a Keras layer that uses a TensorFlow Hub model to the embed sentences, and trying it on some sample input.

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [],
                           dtype = tf.string, trainable = True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

**Building the model on complete dataset**

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Compiling model**

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Train the Text Classification Model**

Train the model for 20 epochs in mini-sets of 512 samples. These are 20 iterations on all samples of the tensors x_train and y_train. During training, monitor model loss and accuracy on the 10,000 samples in the validation set.

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs = 20,
                    validation_data = validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 88ms/step - loss: 1.1745 - accuracy: 0.5009 - val_loss: 0.8324 - val_accuracy: 0.5224
Epoch 2/20
30/30 [==============================] - 4s 122ms/step - loss: 0.7477 - accuracy: 0.5534 - val_loss: 0.6972 - val_accuracy: 0.5885
Epoch 3/20
30/30 [==============================] - 2s 65ms/step - loss: 0.6614 - accuracy: 0.6074 - val_loss: 0.6429 - val_accuracy: 0.6195
Epoch 4/20
30/30 [==============================] - 2s 71ms/step - loss: 0.6134 - accuracy: 0.6483 - val_loss: 0.6006 - val_accuracy: 0.6591
Epoch 5/20
30/30 [==============================] - 3s 81ms/step - loss: 0.5753 - accuracy: 0.6833 - val_loss: 0.5712 - val_accuracy: 0.6856
Epoch 6/20
30/30 [==============================] - 4s 112ms/step - loss: 0.5476 - accuracy: 0.7050 - val_loss: 0.5470 - val_accuracy: 0.7084
Epoch 7/20
30/30 [==============================] - 2s 67ms/step - loss: 0.5203 - accuracy: 0.7271 - val_loss: 0.5234 - val_accuracy: 0.7328
Epoch 8/20


**Evaluating Model**

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 4s - loss: 0.3277 - accuracy: 0.8468 - 4s/epoch - 84ms/step
loss: 0.328
accuracy: 0.847
